##### Thư viện và Hàm

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import re
import os
import numpy as np

In [2]:
#Đọc thư mục chứa file parquet
def Read_parquet_folder(path, from_date = 0, to_date = 20300101):
    list_file = os.listdir(path)
    new_list = []
    for i in range(len(list_file)):
        if int(list_file[i][-16:-8]) >= from_date and int(list_file[i][-16:-8]) <= to_date:
            new_list.append(list_file[i])
        
    df = pd.DataFrame()
    for file in new_list:
        file_path = os.path.join(path, file)
        df_temp = pd.read_parquet(file_path)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [3]:
def get_latest_date(folder_path: str, pattern: str = r"(\d{8})(?=\.parquet$)") -> str:
    pat = re.compile(pattern, re.IGNORECASE)
    latest_date = None

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".parquet"):
            m = pat.search(filename)
            if m:
                d = datetime.strptime(m.group(1), "%Y%m%d")
                if latest_date is None or d > latest_date:
                    latest_date = d

    if not latest_date:
        raise ValueError("Không tìm thấy file phù hợp trong thư mục")

    return latest_date.strftime("%Y%m%d")

##### Code xử lí

In [4]:
now = datetime.now().date()
int_now = int(now.strftime("%Y%m%d"))

to_date = now + timedelta(days=-1)
from_date = to_date + timedelta(days=-6) 
print(from_date, to_date)

2025-09-21 2025-09-27


##### Nhu cầu

In [5]:
df_nhucau = Read_parquet_folder(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-808', from_date = int_now - 14)
df_nhucau = df_nhucau[df_nhucau['Ngành hàng'] == '1254-Thủy Hải Sản Các Loại']
df_nhucau['Ngày nhận hàng'] = pd.to_datetime(df_nhucau['Ngày nhận hàng']).dt.date
df_nhucau = df_nhucau[(df_nhucau['Ngày nhận hàng'] >= from_date) &
                      (df_nhucau['Ngày nhận hàng'] <= to_date)]
if df_nhucau['Ngày nhận hàng'].nunique() != 7:
    raise ValueError("Dữ liệu không đủ 7 ngày")
else:
    print("Dữ liệu đủ 7 ngày")
    df_nhucau['Mã sản phẩm'] = df_nhucau['Tên sản phẩm'].str.extract('(^[0-9]{0,13})').astype('int64')
    df_nhucau['Mã siêu thị'] = df_nhucau['Tên siêu thị'].str.extract('(^[0-9]{0,5})').astype('int64')
    df_nhucau = pd.pivot_table(df_nhucau, index=['Mã siêu thị','Mã sản phẩm'], values='Số lượng cần mua', aggfunc='sum').reset_index()
    df_nhucau = df_nhucau.rename(columns={'Số lượng cần mua': 'SL nhu cầu'})

Dữ liệu đủ 7 ngày


##### Đơn hàng

In [6]:
df_donhang = Read_parquet_folder(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-827', from_date = int_now - 10)
df_product = pd.read_excel(r'D:\ALL FRESH\KHAI BÁO\Danh sách sản phẩm.xlsx', usecols=['Mã sản phẩm','Mã ngành hàng'])
df_donhang = df_donhang.merge(df_product, how='left', on='Mã sản phẩm')
df_donhang = df_donhang[df_donhang['Mã ngành hàng'] == 1254]

df_donhang['Ngày nhập đến'] = pd.to_datetime(df_donhang['Ngày nhập đến']).dt.date
df_donhang = df_donhang[(df_donhang['Ngày nhập đến'] >= from_date) &
                      (df_donhang['Ngày nhập đến'] <= to_date)]
if df_donhang['Ngày nhập đến'].nunique() != 7:
    raise ValueError("Dữ liệu không đủ 7 ngày")
else:
    df_donhang = pd.pivot_table(df_donhang, index=['Mã siêu thị','Mã sản phẩm'], values='Số lượng', aggfunc='sum').reset_index()
    df_donhang = df_donhang.rename(columns={'Số lượng': 'SL lên PO'})

##### Phiếu nhập

In [7]:
df_phieunhap = Read_parquet_folder(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-951', from_date = int_now - 10)
df_phieunhap = df_phieunhap[df_phieunhap['Ngành hàng'] == 'Thủy Hải Sản Các Loại']

df_phieunhap['Ngày nhập'] = pd.to_datetime(df_phieunhap['Ngày nhập']).dt.date
df_phieunhap = df_phieunhap[(df_phieunhap['Ngày nhập'] >= from_date) &
                      (df_phieunhap['Ngày nhập'] <= to_date)]
if df_phieunhap['Ngày nhập'].nunique() != 7:
    raise ValueError("Dữ liệu không đủ 7 ngày")
else:
    df_phieunhap = df_phieunhap[df_phieunhap['Tên hình thức nhập'].isin(["Nhập chuyển kho","Nhập mua nội bộ","Nhập mua hàng (trong nước)"])]
    df_phieunhap = df_phieunhap[(df_phieunhap['Mã khách hàng'] > 10000) |
                                (df_phieunhap['Nhà cung cấp'].str.upper().str.contains('HUB|KHO '))]
    df_phieunhap = pd.pivot_table(df_phieunhap, index=['Mã siêu thị','Mã sản phẩm'], values='Số lượng', aggfunc='sum').reset_index()
    df_phieunhap = df_phieunhap.rename(columns={'Số lượng': 'SL nhập'})

##### Phiếu xuất

In [8]:
df_phieuxuat = Read_parquet_folder(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-735', from_date = int_now - 7)
df_phieuxuat = df_phieuxuat[df_phieuxuat['Ngành hàng'] == 'Thủy Hải Sản Các Loại']

df_phieuxuat['Ngày xuất'] = pd.to_datetime(df_phieuxuat['Ngày xuất']).dt.date
df_phieuxuat = df_phieuxuat[(df_phieuxuat['Ngày xuất'] >= from_date) &
                      (df_phieuxuat['Ngày xuất'] <= to_date)]
if df_phieuxuat['Ngày xuất'].nunique() != 7:
    raise ValueError("Dữ liệu không đủ 7 ngày")
else:
    df_phieuxuat = df_phieuxuat[['Ngày xuất','Mã siêu thị','Mã sản phẩm','Tên sản phẩm','Số lượng','Giá bán','giá bán nguyên giá','Doanh thu theo ngành hàng','Mã hình thức xuất','Ghi chú.1']]

    # region Số lượng xuất
    rules = [
        (
            'SL xuất share hàng',
            (df_phieuxuat['Ghi chú.1'].eq('Yêu Cầu Xin Hàng(MWG)(duyệt yc mới MWG)'))
        ),
        (
            'SL xuất kho chênh lệch',
            (df_phieuxuat['Mã hình thức xuất'].isin([17]))
        ),
        (
            'SL xuất bán',
            (df_phieuxuat['Mã hình thức xuất'].isin([3, 4, 1903]))
        ),
        (
            'SL xuất bán NG',
            (df_phieuxuat['Mã hình thức xuất'].isin([3, 4, 1903])) &
            (df_phieuxuat['Giá bán'] >= df_phieuxuat['giá bán nguyên giá'])
        )
    ]

    for new_col, mask in rules:
        df_phieuxuat[new_col] = np.where(mask, df_phieuxuat['Số lượng'], 0)
    # endregion

    df_phieuxuat = pd.pivot_table(df_phieuxuat, index=['Mã siêu thị','Mã sản phẩm'],
                                  values=['SL xuất share hàng','SL xuất kho chênh lệch','SL xuất bán','SL xuất bán NG','Doanh thu theo ngành hàng'],
                                  aggfunc='sum').reset_index()
    df_phieuxuat.rename(columns={'Doanh thu theo ngành hàng': 'Doanh thu'}, inplace=True)

C:\Users\minhd\AppData\Local\Temp\ipykernel_17120\2290274444.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_temp], ignore_index=True)


In [9]:
df = pd.merge(df_nhucau, df_donhang, how='outer', on=['Mã siêu thị','Mã sản phẩm'])
df = pd.merge(df, df_phieunhap, how='outer', on=['Mã siêu thị','Mã sản phẩm'])
df = pd.merge(df, df_phieuxuat, how='outer', on=['Mã siêu thị','Mã sản phẩm'])
df_product = pd.read_excel(r'D:\ALL FRESH\KHAI BÁO\Danh sách sản phẩm.xlsx',
                           usecols=['Mã sản phẩm','Ngành hàng','Nhóm hàng','Tên sản phẩm','Trọng lượng (kg)'])
df_product['Trọng lượng (kg)'] = df_product['Trọng lượng (kg)'].replace(0, 1)
df_gomcode = pd.read_excel(r'D:\ALL FRESH\KHAI BÁO\Danh sách sản phẩm.xlsx', sheet_name='Gom code',
                           usecols=['Mã sản phẩm','Gom code'])
df = df.merge(df_product, how='left', on='Mã sản phẩm')
df = df.merge(df_gomcode, how='left', on='Mã sản phẩm')
df['SL nhu cầu (KG)'] = (df['SL nhu cầu'] * df['Trọng lượng (kg)'])
df['SL lên PO (KG)'] = (df['SL lên PO'] * df['Trọng lượng (kg)'])
df['SL nhập (KG)'] = (df['SL nhập'] * df['Trọng lượng (kg)'])
df['SL xuất share hàng (KG)'] = (df['SL xuất share hàng'] * df['Trọng lượng (kg)'])
df['SL xuất kho chênh lệch (KG)'] = (df['SL xuất kho chênh lệch'] * df['Trọng lượng (kg)'])
df['SL xuất bán (KG)'] = (df['SL xuất bán'] * df['Trọng lượng (kg)'])
df['SL xuất bán NG (KG)'] = (df['SL xuất bán NG'] * df['Trọng lượng (kg)'])

df['SL thực nhập (KG)'] = df['SL nhập (KG)'] - df['SL xuất kho chênh lệch (KG)'] - df['SL xuất share hàng (KG)']
df = df[['Ngành hàng','Nhóm hàng','Gom code','Tên sản phẩm','Mã siêu thị','SL nhu cầu (KG)','SL lên PO (KG)','SL thực nhập (KG)','SL xuất bán (KG)']]
df = pd.pivot_table(df, index=['Ngành hàng','Nhóm hàng','Gom code','Mã siêu thị'],
                    values=['SL nhu cầu (KG)','SL lên PO (KG)','SL thực nhập (KG)','SL xuất bán (KG)'],
                    aggfunc=sum).reset_index()

df = df[df['SL lên PO (KG)'] > 0]
df = df[df['SL thực nhập (KG)'] > 0]
df['Tỉ lệ nhập/PO'] = ((df['SL thực nhập (KG)'] / df['SL lên PO (KG)']) * 100).round(0).astype(int).astype(str) + '%'
df['Tỉ lệ bán/nhập'] = ((df['SL xuất bán (KG)'] / df['SL thực nhập (KG)']) * 100).round(0).astype(int).astype(str) + '%'
df = df.sort_values(by=['Nhóm hàng','SL thực nhập (KG)'], ascending=[True,False])
df = df.rename(columns={'Gom code': 'Nhóm sản phẩm'})
df['Từ ngày'] = from_date.strftime("%d/%m/%Y")
df['Đến ngày'] = to_date.strftime("%d/%m/%Y")

df = df.rename(columns={'SL nhu cầu (KG)': 'Nhu cầu',
                        'SL lên PO (KG)': 'PO',
                        'SL thực nhập (KG)': 'Nhập',
                        'SL xuất bán (KG)': 'Bán',
                        'Tỉ lệ nhập/PO': '% Nhập/PO',
                        'Tỉ lệ bán/nhập': '% Bán/Nhập'
                        }
)
df['Nhu cầu'] = df['Nhu cầu'].round(1)
df['PO'] = df['PO'].round(1)
df['Nhập'] = df['Nhập'].round(1)
df['Bán'] = df['Bán'].round(1)

df.head()

C:\Users\minhd\AppData\Local\Temp\ipykernel_17120\3329486866.py:21: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df = pd.pivot_table(df, index=['Ngành hàng','Nhóm hàng','Gom code','Mã siêu thị'],


,Ngành hàng,Nhóm hàng,Nhóm sản phẩm,Mã siêu thị,PO,Nhu cầu,Nhập,Bán,% Nhập/PO,% Bán/Nhập,Từ ngày,Đến ngày
2935,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA ĂN LIỀN,4340,16.8,16.8,16.8,4.9,100%,29%,21/09/2025,27/09/2025
1989,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4484,13.3,2.8,14.0,10.5,105%,75%,21/09/2025,27/09/2025
2066,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4783,13.3,13.3,13.0,11.6,97%,89%,21/09/2025,27/09/2025
2053,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4749,12.6,12.6,12.6,8.0,100%,64%,21/09/2025,27/09/2025
2342,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,6520,12.6,12.6,12.6,10.2,100%,81%,21/09/2025,27/09/2025


##### Gộp dữ liệu số mua hiện tại

In [10]:
df_muadeu = pd.read_parquet(r'D:\LINE chatbot\data.parquet')
df_muadeu = df_muadeu[['Tên sản phẩm','Mã siêu thị','Trạng thái chia hàng']]
df_gomcode = pd.read_excel(r'D:\ALL FRESH\KHAI BÁO\Danh sách sản phẩm.xlsx', usecols=['Tên sản phẩm','Gom code'], sheet_name='Gom code')
df_muadeu = df_muadeu.merge(df_gomcode, how='left', on='Tên sản phẩm')
df_muadeu = df_muadeu.rename(columns={'Trạng thái chia hàng': 'Số chia hiện tại', 'Gom code': 'Nhóm sản phẩm'})
df = df.merge(df_muadeu[['Mã siêu thị','Nhóm sản phẩm','Số chia hiện tại']], how='left', on=['Mã siêu thị','Nhóm sản phẩm'])
df.head()

,Ngành hàng,Nhóm hàng,Nhóm sản phẩm,Mã siêu thị,PO,Nhu cầu,Nhập,Bán,% Nhập/PO,% Bán/Nhập,Từ ngày,Đến ngày,Số chia hiện tại
0,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA ĂN LIỀN,4340,16.8,16.8,16.8,4.9,100%,29%,21/09/2025,27/09/2025,Sức bán
1,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4484,13.3,2.8,14.0,10.5,105%,75%,21/09/2025,27/09/2025,Min
2,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4783,13.3,13.3,13.0,11.6,97%,89%,21/09/2025,27/09/2025,Sức bán
3,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,4749,12.6,12.6,12.6,8.0,100%,64%,21/09/2025,27/09/2025,Min
4,Thủy Hải Sản Các Loại,CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA CẮT SỢI,6520,12.6,12.6,12.6,10.2,100%,81%,21/09/2025,27/09/2025,Sức bán


In [11]:
df.reset_index().to_parquet('data_nhapban.parquet')